In [11]:
##### #03/22/2023 Xilin found out it used the last min_action no matter what - revised it
import scipy
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations
import time
import random
random.seed(10)
from scipy.stats import beta
import pandas as pd
import copy
%matplotlib inline
#%run Helpers.ipynb
#%run pure_strategy_selection.ipynb  #include simple selection algorithm
import scipy.io
import collections
import sys
import itertools
%run Synthetic.ipynb

## Mathmatic Functions

In [14]:
# centers the opinion vector around 0\n",
def mean_center(op, n):
    ones = np.ones((n, 1))
    x = op - (np.dot(np.transpose(op),ones)/n) * ones
    return x
    
# compute number of edges, m\n
def num_edges(L, n):
    m = 0
    for i in range(n):
        for j in range(n):
            if i > j and L[i,j] < 0:
                m += 1            
    return m

# maximizing polarization only: \\bar{z}^T \\bar{z}   
def obj_polarization(A, op, n):
    op_mean = mean_center(op, n)
    z_mean = np.dot(A, op_mean) 
    return np.dot(np.transpose(z_mean), z_mean)[0,0] 

def obj_polarization_1(A, op, n):
    z = np.dot(A, op) 
    z_mean = mean_center(z, n)
    return np.dot(np.transpose(z_mean), z_mean)[0,0] 

# Calculate innate polarization
def obj_innate_polarization(s, n):  
#     np.set_printoptions(precision=5)
    op_mean = mean_center(s, n)
    return np.dot(np.transpose(op_mean), op_mean)[0,0]

In [16]:
# Parameters for the network
np.set_printoptions(precision=3)
# n = 20

## Creating Network
### 1. Make Random Network

In [40]:
def create_erdos_renyi_network(n, p, u, v):
    A = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
#         for j in range(i+1,n):
#             r = np.random.rand()
#             if r < p:
            if j<i:
                A[i,j] = A[j,i] = 1
#             if i==u or i==v:
#                 A[i,j] = A[j,i] = 1
    return A


def make_innat_opinions(n): # Make opinion for agents only - no info source is involved
    
    # Make list of ind innate opinion to define info source opinion
    innat_s = np.random.uniform(low=0, high=1, size=int(n))   #individual's innate opinion 

    s = np.zeros((n, 1))
    
    idx1 = 0
    for i in range(len(s)):
        s[i] = innat_s[idx1]  #set innate opinion for ind.
        idx1 += 1
  
    return s

def make_random_network(n):
    # Create empty graph
    nxG = nx.Graph()
    # Add nodes to graph
    nxG.add_nodes_from(range(n))
    # Add edges based on node index
    for i in range(n):
        for j in range(i):
            p = (n-i)/(n-j) # Probability of edge existing
            if np.random.random() < p:
                nxG.add_edge(i, j)

    # Draw graph
#     nx.draw(nxG, with_labels=True)
#     plt.show()
        # Check for isolated nodes and add one connection if needed
    isolated_nodes = list(nx.isolates(nxG))
    if isolated_nodes:
        non_isolated_nodes = [node for node in nxG.nodes if node not in (isolated_nodes)]
        random_node = np.random.choice(non_isolated_nodes)
        nxG.add_edge(isolated_nodes[0], random_node)

    # Draw graph
    # nx.draw(nxG, with_labels=True)
    # plt.show()
    G = nx.adjacency_matrix(nxG).todense()
    
    return G



In [42]:
def visualize(s,G):
    # what the twitter graph looks like 
    s_use = s.flatten()   # Convert array to a list for later operation
    s_use = s_use.tolist()
    new_s = [i * 30 for i in s_use]
    df = pd.DataFrame(new_s, columns=['Opinion']) #create a datafram with index at column 1, opinion at column 2

    nxG = nx.from_numpy_matrix(G)  
   
    #plt.figure(figsize=(10, 10))
        ##### Calculate Key Values  ######
    L = scipy.sparse.csgraph.laplacian(G, normed=False)  # Return the Laplacian matrix
    A = np.linalg.inv(np.identity(n) + L)  # A = (I + L)^(-1)\n  Stanford paper theory
    columnsum_ij = np.sum(A, axis=0)
    m = num_edges(L, n)                    # call the function to calculate the number of edges

#     print(n)
    # what the twitter graph looks like 

    def node_edge(G, n):
        edges =[]
        for v in range(n):
            a = np.array(np.nonzero(G[v])[0])
            edge = len(a)
    #         print(edge)
            edges.append(edge)

        return edges

    node_edges = node_edge(G, n)
    # print(node_edges)

    node_sizes =[]
    for i in node_edges:
        node_size = 1/i*8000
        node_sizes.append(node_size)
        
#     plt.figure(figsize=(20, 20))

    # Fix seed - fix network shape
    my_pos = nx.spring_layout(nxG, seed = 2)
    nx.draw(nxG, pos= my_pos, with_labels= True, node_color=df['Opinion'].astype(int),cmap=plt.cm.Blues, node_size= node_sizes, edge_color='black', width=0.8, font_color='black',font_size=26, font_weight='bold', alpha=0.8)
    #nx.draw(nxG, pos = my_pos, with_labels=False, node_color=color_map, node_size= node_sizes, edge_color='grey', width=0.5, font_color='white',font_size=9, font_weight='bold')
    sm = plt.cm.ScalarMappable(cmap=plt.cm.Blues, norm=plt.Normalize(vmin = 0, vmax=1))
    cbar = plt.colorbar(sm, shrink = 0.5)
    tick_font_size = 24
    cbar.ax.tick_params(labelsize=tick_font_size)
   # plt.show()
    


# if we want to customize the color bar range to min/max s
# vmin = min(s)
# vmax = max(s)
# sm = plt.cm.ScalarMappable(cmap=plt.cm.Blues, norm=plt.Normalize(vmin = vmin, vmax=vmax))
# sm._A = []
# plt.colorbar(sm,shrink=0.5)
# plt.show()

### Min [245, ] [284, ]       Max [50, ] [481, ] 

### 4. Equilibrium & Polarization  - based on derivation
$$P(z) = z ^T * z $$


In [46]:
# op = s
# y = mean_center(s,n)
# # print(y)
# innat_pol = np.dot(np.transpose(y), y)[0,0] 
# print('Innate_polarization:')
# print(innat_pol)

# # Test equilibrium polarization
# equ_pol = obj_polarization(A, s, n)
# print('Equi_polarization:')
# print(equ_pol)

# di = equ_pol-innat_pol
# print("Difference:")
# print(di)

In [63]:
# # Function call and handling its output
# output = choose_max_vertex(s, n, A)
# print(output)
# if len(output) == 4:
#     v1, max_opinion, max_pol, min_action = output
# else:
#     v1, max_opinion, max_pol = output
#     min_action = None  # Default or error handling value

# # Continue with your logic
# (v2, min_opinion) = min_action if min_action else (None, None)
# if v1 is None:  # Handle the case where maximizer cannot find a valid action
#     pass  # Some fallback or error handling logic


(19, 1, 0.03339206699032296, (4, 0.9115513774010708))


### Testing players' behavior

In [49]:
def MaxMin_play(s,n, G):    # maxmizer first-time play, greedy algorithm
    #print('Maximizer Play')
    ##### Calculate Key Values  ######
    L = scipy.sparse.csgraph.laplacian(G, normed=False)  # Return the Laplacian matrix
    A = np.linalg.inv(np.identity(n) + L)  # A = (I + L)^(-1)\n  Stanford paper theory
    columnsum_ij = np.sum(A, axis=0)
    #print(columnsum_ij)
    champion = choose_max_vertex(s, n,A) # The best choice among all opinions and vertexs, function is in "pure_strategy_selection.ipynb"
   # print('champion',champion)
    (v1, max_opinion, max_pol, min_action) = champion
    (v2, min_opinion) = min_action
    if v1 == None:   # if maximizer cannot find one
        print('Maximizer fail')
    else:
        # print("                                ")
        # print("Maximizer finds its target agent:")
#         print('v1', 'changed_opinion', 'innate_obj', 'obj')
#         print(max_champion)

        #Store innate_op of the max_selected vertex
        op = copy.copy(s)
        old_opinion_max = op[v1, 0]
        old_opinion_min = op[v2, 0]
        ##### change the agent's opinion with best action(agent v1, max_op)

        ## check if agent's opinionis is changed or not
        # print("Max Action:    "+"Agent" + str(v1) +" 's opinion " + str(old_opinion_max) + " changed to "+ str(max_opinion))
        # print("Min Action:    "+"Agent" + str(v2) +" 's opinion " + str(old_opinion_min) + " changed to "+ str(min_opinion))
        # print("Network reaches equilibrium Polarization: " + str(max_pol))


    return(v1, v2, max_opinion, min_opinion, max_pol)



# determines if value of opinion at v should be set to 0 or 1 to maximize equilibrium polarization 
def get_max_opinion(s, n, v1,A):
    
    por_arr = np.zeros(2)  # create a two_element array to store polarization value of each option
    max_opi_option = [0, 1.0]   # Maximizer has two options to change agent v1's opinion
    min_actions = []
    # objective if set opinion to 0, 1.0
    j = 0
    for max_opinion in max_opi_option:
        (v2,min_opinion, min_pol) = maxmin_polar(s,v1,max_opinion,A)
        por_arr[j] = min_pol
        j = j + 1   # index increase 1, put the polarization in array
        min_actions.append((v2,min_opinion))

    maxmize_op = np.argmax(por_arr)  # the index of maximum polarization = max_opinion --[0,1]
    max_por = np.max(por_arr)        # find the maximum polarization in the record
    #03/22/2023 Xilin found out it used the last min_action no matter what
    min_action = min_actions[np.argmax(por_arr)]  # the index of maximum polarization --> optimal min action 
    
    
    return (maxmize_op, max_por, min_action)

# determine which agent maximizer should select to maximizer the equilibrium polarization
def choose_max_vertex(s, n,A):
   # max_por = obj_polarization(A, op, n)  # use "innate"(after min action) polarization as a comparable standard to find max_por
    max_por = 0
    C1 = list(range(n))    # for all agent 
    for v1 in C1:         
            #print('Maximizer start from agent'+str(v1))
            (max_opinion, por, min_action) = get_max_opinion(s, n, v1,A)
            #print('min_action',min_action)
            if por > max_por: # if the polarization of most recent action > maximum polarization of previous actions
                max_por = por
                champion = (v1, max_opinion,max_por, min_action)   # save the this action as champion    

    return (champion)


def maxmin_polar(s, v1, max_opinion,A):
    op = copy.copy(s)
    op[v1] = max_opinion  
    (v2,min_opinion, min_pol) = minimizer_play(op,n,v1,A)
    
    return (v2,min_opinion, min_pol)
    
      
##### minimizer first-time play, greedy algorithm
def minimizer_play(op,n,max_touched,A): 
    
    op1 = copy.copy(op)
    min_champion = choose_min_vertex(op1, n, max_touched,A) 
    (v2, min_opinion, min_pol) = min_champion
    
    if v2 == None:
        print('Minimizer fail')

    return (v2,min_opinion, min_pol)


def derivate_s(op,n,v2,A):
               #op - opinion array that updated by maximizer
    c = [1/n] * n
#     print(c)
    sum_term = 0
    j = 0

    sum_term = np.dot(np.dot((A-c),(A[v2]-c)),op)  # sum up all terms
    
    term_out = op[v2]*np.dot((A[v2]-c),(A[v2]-c)) # exclude the term that j = v2
    sum_s = sum_term - term_out    # numerator
    
    s_star = -sum_s/np.dot((A[v2]-c),(A[v2]-c))
    s_star = s_star[0] #take value out of array
#     print(str(v2)+'s_star:'+str(s_star))
    #min_opinion =min(max(0,s_star),1)
    bound = (0,1)
    por_arr = []  # create a two_element array to store polarization value of each option
    
    if s_star<0 or s_star>1:
        for min_opinion in bound:
            #change max_opinion array and calculate the polarization
            op1 = copy.copy(op)
            op1[v2] = min_opinion   #after max action, update min action on opinion array
#             print('op1:', op1)
            min_por = obj_polarization(A, op1, n)
            por_arr.append(min_por)
#         print("por_arr:", por_arr)
        minmize_op = np.argmin(por_arr)  # the index of maximum polarization = max_opinion --[0,1]
        min_por = np.min(por_arr)        # find the maximum polarization in the record    
    else:
        minmize_op = s_star
        op1 = copy.copy(op)
        op1[v2] = minmize_op   #after max action, update min action on opinion array
#         print('op1:', op1)
        min_por = obj_polarization(A, op1, n)
        
    return (minmize_op, min_por)
     

    # Minimizer search: Go through each agent 

def choose_min_vertex(op, n, max_touched, A):
    # current opinion array that changed by maximizer, "innate" opinion that min start with

    champion = (None, None, 0, None)  # assume the best action is champion
    min_por = 10000
    all = list(range(n))    # for all agent 
    C1 = [x for x in all if x != max_touched]  # for the vertice that Maximizer has not touched
    
    for v2 in C1:   
        #print('Min start with agent '+ str(v2) )
        (changed_opinion, por) = derivate_s(op,n,v2,A)   # find the best new_op option           

        if por < min_por:  # if the recent polarization is smaller than the minimum polarization in the history
            min_por = por
                                 # update the recent option as champion
            champion = (v2, changed_opinion, min_por)  
    #print("champion:", champion)
    return (champion)  # find the best minimizer's action after going through every new_op option of every agent


### Minimizer Strategy

## Network Analysis

In [53]:
def normalize_dict(converted_dict):
    
    X = np.array(list(converted_dict.values()))
    x_mean = X.mean
    XNormed = (X - X.mean())/(X.std())

    i = 0
    for key in converted_dict.keys():
        converted_dict[key]= XNormed[i]
        i = i+1
    
    return(converted_dict)


def net_rank(s,G, normalize):
    #print("___________________Max Analyze__________________________________________")
    nxG = nx.from_numpy_matrix(G) 
    # G = nx.karate_club_graph()
    #print("_______________Degree Centrality_____________________________")  
    deg_centrality = nx.degree_centrality(nxG)
    sortedDict = sorted(deg_centrality.items(), key=lambda x:x[1])
    converted_dict = dict(sortedDict)
    # print(converted_dict)
    #print("                           ")
    #print("_______________Closeness Rank_____________________________")
    close_centrality = nx.closeness_centrality(nxG)
    sortedDict1 = sorted(close_centrality.items(), key=lambda x:x[1])
    converted_dict1 = dict(sortedDict1)
    # print(converted_dict1)
    #print("                           ")
    #print("_______________Page Rank_____________________________")
    pr = nx.eigenvector_centrality(nxG)
    sortedDict2 = sorted(pr.items(), key=lambda x:x[1])
    converted_dict2 = dict(sortedDict2)
    
    # print(converted_dict2)

    #print("                           ")

    def gap(op, n):
        ones = np.ones((n, 1))
        x = op - (np.dot(np.transpose(op),ones)/n) * ones
        return abs(x)

    gap = gap(s,n)
    my_gap = {index: value for index, value in enumerate(gap)}
    sorting_gap = sorted(my_gap.items(), key=lambda x:x[1], reverse=True)
    sorted_gap = dict(sorting_gap)
#     print("opinion - mean")
#     print(sorted_gap)
    if normalize == True:
        converted_dict = normalize_dict(converted_dict)
        converted_dict1 = normalize_dict(converted_dict1)
        converted_dict2 = normalize_dict(converted_dict2)
        sorted_gap = normalize_dict(sorted_gap)
    else:
        pass
        
    return (converted_dict,converted_dict1,converted_dict2,sorted_gap)

def pred_error(s, G, max_pol, v1, min_pred):
    op = copy.copy(s)
    op[v1] = max_opinion  # change max's action 
    op1 = copy.copy(op)
    op1[min_pred] = np.mean(s) #change predicted min's node to mean opinion
    L = scipy.sparse.csgraph.laplacian(G, normed=False)  # Return the Laplacian matrix
    A = np.linalg.inv(np.identity(n) + L)  # A = (I + L)^(-1)\n  Stanford paper theory
    pred_por = obj_polarization(A, op1, n)
    dif = max_pol-pred_por
    print("difference in prediction:"+str(abs(max_pol-pred_por)))
    return dif



def pred_old(N,converted_dict,converted_dict1,converted_dict2,sorted_gap):
    # Using islice() + items()
    # Get first N items in dictionary
    a = list(itertools.islice(converted_dict.keys(), N))
    b = list(itertools.islice(converted_dict1.keys(), N))
    c = list(itertools.islice(converted_dict2.keys(), N))
    
    max_pred = c
    #max_pred = sorted(np.unique(a+b+c))
    min_pred = list(itertools.islice(sorted_gap.keys(), N))
 
#     #print("*********Manual new ranks*************")      
#     new_rank = {i: 0.5*op_ranks.get(i, 0) + 0.5*ecen_ranks.get(i, 0) for i in set(op_ranks).union(ecen_ranks)}
#     #print(new_rank)
#     sorting_rank = sorted(new_rank.items(), key=lambda x:x[1])
#     sorted_rank = dict(sorting_rank)
# #     print(sorted_rank)
#     min_pred1= list(itertools.islice(sorted_rank.keys(), N))
    
#     first_key = next(iter(converted_dict2)) 
#     converted_dict2.pop(first_key)
#     min_pred2= list(itertools.islice(converted_dict2.keys(), N))

#     min_pred = min_pred1 + min_pred2
# #     print(min_pred)

#     #printing result
# #     print("Min limited by K is : " + str(min_pred))
    
    return (max_pred, min_pred)



In [55]:
# (v1,v2,max_pol) = MaxMin_play(s,n)
def run(n, nomalize):
    s = make_innat_opinions(n)
    G = make_random_network(n)
    (a,b,c,d)= net_rank(s,G, normalize)
#   visualize(s,G)
    (v1, v2, max_opinion, min_opinion, max_pol) = MaxMin_play(s,n,G)
    
    return (v1,v2, max_opinion, min_opinion,max_pol, s, G, a,b,c,d)


In [59]:
# n = 20
# s,G,A = make_sync_network(n)
# MaxMin_play(s,n, G)


                                
Maximizer finds its target agent:


(19, 4, 1, 0.9115513774010708, 0.03339206699032296)